# Lumped BDDC Example

In [ ]:
# dependencies
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.develop(path="../..")
using Plots
using LFAToolkit
using LinearAlgebra

In [ ]:
# setup
p = 1
mesh = Mesh2D(1.0, 1.0)
numberelements1d = 4

# operator
diffusion = GalleryMacroElementOperator("diffusion", p + 1, p + 2, numberelements1d, mesh);

# bddc smoother
bddc = LumpedBDDC(diffusion)

In [ ]:
# full operator symbols
numbersteps = 150
θ_min = -π/2
θ_max = 3π/2
θ_range = LinRange(θ_min, θ_max, numbersteps)

# compute and plot smoothing factor
# setup
ω = [0.2]
maxeigenvalues = zeros(numbersteps, numbersteps)

# compute
for i in 1:numbersteps, j in 1:numbersteps
    θ = [θ_range[i], θ_range[j]]
    if sqrt(abs(θ[1])^2 + abs(θ[2])^2) > π/128
        A = computesymbols(bddc, ω, θ)
        eigenvalues = [abs(val) for val in eigvals(I-A)]
        maxeigenvalues[i, j] = maximum(eigenvalues)
    end
end

# plot
println("max eigenvalue: ", maximum(maxeigenvalues))
xrange = θ_range/π
heatmap(
    xrange,
    xlabel="θ/π",
    xtickfont=font(12, "Courier"),
    xrange,
    ylabel="θ/π",
    ytickfont=font(12, "Courier"),
    maxeigenvalues,
    title="Lumped BDDC Error Symbol",
    transpose=true,
    aspect_ratio=:equal
)
xlims!(θ_min/π, θ_max/π)
ylims!(θ_min/π, θ_max/π)